In [1]:
import sys 
import os
import argparse
sys.path.append('./project/')
sys.path.append(f'/users/Etu2/3701222/.local/lib/python3.7/site-packages')


import pytorch_lightning as pl
from vision_transformer_org import VisionTransformer
from lightning_modules import CIFAR10DataModule, LitClassifierModel, load_from_checkpoint
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor


In [2]:
! mkdir -p /tempory/vit

In [3]:
directory = "/tempory/vit"
"""
args = argparse.Namespace(
    ##########useful args
    fit=True,
    default_root_dir=f"{directory}/model",
    data_path=f"{directory}/cifar10",
    gpus=-1,
    ########Data Args##########
    image_size=224,
    num_classes=10,
    ########Training Args##########
    learning_rate=1e-4,
    val_size=0.2,
    batch_size=32,
    num_workers=14,
    #######Model args############
    patch_size=32,
    emb_dim=768,
    mlp_dim=3072,
    num_heads=12,
    num_layers=12,
    attn_dropout_rate=0.0,
    dropout_rate=0.1,
    ########Trainer Args##########
    progress_bar_refresh_rate=25,
)
"""
args = argparse.Namespace(
    ##########useful args
    fit=True,
    default_root_dir=f"{directory}/model",
    data_dir=f"{directory}/cifar10",
    gpus=-1,
    ########Data Args##########
    image_size=224,
    num_classes=10,
    ########Optimization Args##########
    learning_rate=1e-4,
    weight_decay=0.01,
    ########Training Args##########
    val_size=0.2,
    batch_size=16,
    num_workers=14,
    #######Model args############
    patch_size=16,
    emb_dim=768,
    mlp_dim=3072,
    num_heads=12,
    num_layers=12,
    attn_dropout_rate=0.0,
    dropout_rate=0.1,
    embedding_mode="linear",
    ########Trainer Args##########
    progress_bar_refresh_rate=25,
)
load =True

In [4]:
datamodule = CIFAR10DataModule(**vars(args))

vit_Backbone = VisionTransformer(**vars(args))

checkpoint_callback = ModelCheckpoint(
        monitor='val_acc',
        filename='vit-{epoch:02d}-{val_loss:.2f}-{val_acc:.2f}',
        mode='max',
)
lr_monitor = LearningRateMonitor(logging_interval='step')


In [5]:
trainer = pl.Trainer.from_argparse_args(args, callbacks=[checkpoint_callback, lr_monitor])

GPU available: True, used: True
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [6]:
if not load:
    model = LitClassifierModel(vit_Backbone, **vars(args))
else:
    model = LitClassifierModel(load_from_checkpoint(
                                VisionTransformer, 
                                LitClassifierModel, 
                                hparams_file="/tempory/vit/model/lightning_logs/version_12/hparams.yaml",
                                checkpoint_file="/tempory/vit/model/lightning_logs/version_12/checkpoints/vit-epoch=16-val_loss=1.11-val_acc=0.74.ckpt",
                                ).backbone,
    **vars(args)
                              ) 

In [ ]:
trainer.fit(model, datamodule)

Files already downloaded and verified
Files already downloaded and verified



  | Name      | Type               | Params
-------------------------------------------------
0 | criterion | LabelSmoothingLoss | 0     
1 | backbone  | VisionTransformer  | 85.8 M
-------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params

  | Name      | Type               | Params
-------------------------------------------------
0 | criterion | LabelSmoothingLoss | 0     
1 | backbone  | VisionTransformer  | 85.8 M
-------------------------------------------------
85.8 M    Trainable params
0         Non-trainable params
85.8 M    Total params


In [ ]:
trainer.test(model, datamodule=datamodule)